In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import re
import os
import time

In [2]:
stop_words = set(stopwords.words('english'))

In [3]:
dictionary = []

In [4]:
df = pd.read_csv("data/dataset/train-balanced-sarcasm.csv")

In [5]:
df_new = df[['parent_comment','comment','label']]

In [6]:
df_new = df_new.sample(100000)

In [7]:
df_new.shape

(100000, 3)

In [8]:
df_new.head()

,parent_comment,comment,label
366594,I got Brazilianed,That'll teach you to Lagswitch!,1
399687,Can we get a 3rd grade class to sit down with ...,"Woah, hey man, I don't know if you know this, ...",1
815782,(Serious) Will there be texting and ways to me...,Noooo everyone is just going to stop using and...,1
566824,Vatican sacks priest after he comes out as gay...,But the pope has made huge changes to the church,1
575551,Detroit BBQ Company owner under fire for socia...,"Wow, he sounds like such a delightful person.",1


In [9]:
def remove_stopwords(tokens):
    tokens_wo_stopwords = []
    for i in range(0,len(tokens)):
        if tokens[i].lower() not in stop_words:
            tokens_wo_stopwords.append(tokens[i].lower())
    return tokens_wo_stopwords

In [10]:
def get_pos_tag(token):
    pos_tag = nltk.pos_tag([token])[0][1]
    if pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [11]:
def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    for i in range(0,len(tokens)):
        tokens[i] = lemmatizer.lemmatize(tokens[i],pos=str(get_pos_tag(tokens[i])))
    return tokens

In [12]:
def add_to_dictionary(tokens):
    for token in tokens:
        if token not in dictionary:
            dictionary.append(token)

In [17]:
def save_dictionary():
    with open('data/processed/dictionary.txt','w') as file:
        file.writelines("%s\n" % word for word in dictionary)

In [18]:
def create_dictionary(dataset):
    for index,row in dataset.iterrows():
        processed_parent_comment = re.sub(r'[^a-zA-Z]', ' ', str(row['parent_comment']))
        processed_comment = re.sub(r'[^a-zA-Z]', ' ', str(row['comment']))
        comment = processed_parent_comment + " " + processed_comment
        tokens_comment = word_tokenize(comment)
        tokens_comment = remove_stopwords(tokens_comment)
        tokens_comment = lemmatize(tokens_comment)
        add_to_dictionary(tokens_comment)
    save_dictionary()

In [19]:
if not os.path.isfile('data/processed/dictionary.txt'):
    starttime = time.time()
    create_dictionary(df_new)
    endtime = time.time()
    print("Time to create dictionary")
    print(endtime - starttime)

Time to create dictionary
508.1313588619232


In [21]:
len(dictionary)

67398

In [32]:
vocab_dictionary = []
with open('data/processed/dictionary.txt', 'r') as f:
    vocab_dictionary = [line.replace('\n','') for line in f]
word2int = {}
int2word = {}
vocab_size = len(vocab_dictionary) # gives the total number of unique words
for i,word in enumerate(vocab_dictionary):
    word2int[word] = i
    int2word[i] = word

In [36]:
all_sentences = []
for index,row in df_new.iterrows():
        processed_parent_comment = re.sub(r'[^a-zA-Z]', ' ', str(row['parent_comment']))
        processed_comment = re.sub(r'[^a-zA-Z]', ' ', str(row['comment']))
        all_sentences.append(processed_parent_comment.split())
        all_sentences.append(processed_comment.split())
print(all_sentences)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

